# Packet analysis of a TCP Network

Using the SUEE1 dataset, we will analyse TCP packet data of a 'eduroam' network and build three machine learning classifiers with the objective of identifying IPs running slowloris attack inside the network. 

In [1]:
# Importando bibliotecas necessárias

import os


import pandas as pd





In [2]:
# Setting global vars

DATASETS_PATH = "datasets/"

In [15]:
# Reading the original, unchanged dataset after .CSV export

original_df = pd.read_csv(os.path.join(DATASETS_PATH, "original_dataset.csv"), sep=',', index_col="frame.number")

In [16]:
# Number of lines inside the dataset

print("Dataset number of lines: {}".format(len(original_df)))

Dataset number of lines: 2089436


In [17]:
# A little look at our dataset

original_df.head()

,frame.len,frame.protocols,eth.src,eth.dst,ip.proto,ip.len,ip.src,ip.dst,ip.ttl,tcp.flags,tcp.seq_raw,tcp.seq,tcp.stream,tcp.window_size_value,tcp.time_relative,tcp.time_delta
frame.number,,,,,,,,,,,,,,,,
1,66,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:01,ff:ff:ff:ff:00:02,6,52,192.168.0.1,192.168.0.2,64,0x00000011,1202351615,1,0,235,0.000000,0.000000
2,66,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:03,ff:ff:ff:ff:00:01,6,52,192.168.0.2,192.168.0.1,62,0x00000010,2705104314,1,0,245,0.038679,0.038679
3,54,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:01,ff:ff:ff:ff:00:02,6,40,192.168.0.1,192.168.0.3,64,0x00000011,3098131932,1,1,237,0.000000,0.000000
4,54,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:03,ff:ff:ff:ff:00:01,6,40,192.168.0.3,192.168.0.1,126,0x00000010,1558776769,1,1,254,0.001364,0.001364
5,66,eth:ethertype:ip:tcp,ff:ff:ff:ff:00:01,ff:ff:ff:ff:00:02,6,52,192.168.0.1,192.168.0.4,64,0x00000011,1199048573,1,2,235,0.000000,0.000000


In [11]:
# Columns and their respective type

original_df.dtypes

frame.number               int64
frame.len                  int64
frame.protocols           object
eth.src                   object
eth.dst                   object
ip.proto                   int64
ip.len                     int64
ip.src                    object
ip.dst                    object
ip.ttl                     int64
tcp.flags                 object
tcp.seq_raw                int64
tcp.seq                    int64
tcp.stream                 int64
tcp.window_size_value      int64
tcp.time_relative        float64
tcp.time_delta           float64
dtype: object

In [21]:
# Statistical analysis of the quantitative columns

original_df[['frame.len',
             'ip.len',
             'ip.ttl',
             'tcp.window_size_value',
             'tcp.time_relative',
             'tcp.time_delta' 
           ]].describe()

,frame.len,ip.len,ip.ttl,tcp.window_size_value,tcp.time_relative,tcp.time_delta
count,2.089436e+06,2.089436e+06,2.089436e+06,2.089436e+06,2.089436e+06,2.089436e+06
mean,6.234917e+01,7.901674e+02,6.643968e+01,1.011751e+04,-2.929557e+01,-5.852291e-01
std,9.250634e+00,2.815735e+03,1.859239e+01,1.862629e+04,3.708226e+02,6.531709e+01
min,5.400000e+01,4.000000e+01,3.700000e+01,0.000000e+00,-4.293922e+03,-4.293949e+03
25%,5.400000e+01,4.400000e+01,6.200000e+01,2.350000e+02,1.067800e-02,7.000000e-06
50%,6.600000e+01,5.200000e+01,6.400000e+01,3.050000e+02,2.354410e-01,4.300000e-05
75%,6.600000e+01,5.570000e+02,6.400000e+01,8.192000e+03,1.545133e+00,1.274000e-03
max,5.370000e+02,6.521200e+04,2.540000e+02,6.553500e+04,2.689156e+03,2.684000e+03


In [32]:
# Checking how many unique IP addresses there's on the dataset

print("IP sources: {}\n"\
       "IP destinations: {}".format(len(original_df['ip.src'].value_counts()), len(original_df['ip.dst'].value_counts())))

IP sources: 1859
IP destinations: 1858


## Data cleaning

In [ ]:
# Discarding useless columns



